# Product Recommender System

The Recommender System utilizes the Chinese Natural Language Processing technique to suggest products with similar product descriptions.  

'Jieba' , a popular Chinese toekenization library was used, also Both jacard similarity and coseine distance were incorporated to generate KNNs. 

In [4]:
import pandas as pd 
import json
from pandas.io.json import json_normalize
from nltk.tokenize import RegexpTokenizer
from collections import Counter
import jieba

In [5]:
file = 'product_content.json'
with open(file) as f:
    prod_df = json.load(f,strict=False)

In [6]:
prod_df=pd.io.json.json_normalize(prod_df,'records')
prod_df.head()

,city_cd,country_cd,hot_sort,main_cat,prod_desc,prod_oid,property_type
0,A01-014-00003,A01-014,0,M06,－ 開始時間：7:00am 或 4:00pm\n－ 出發地點：您於暹粒的下榻飯店\n－ 行程...,9161,DESC
1,A01-004-00001,A01-004,0,M06,"[{""tips"":{""checked"":""N""}},{""ins"":{""checked"":""N...",10689,INC_DATA
2,A01-004-00007,A01-004,0,M06,"[{""tips"":{""checked"":""N""}},{""ins"":{""checked"":""N...",10689,INC_DATA
3,A01-007-00003,A01-007,0,M06,【晚霞美酒】曼德勒烏本橋日落＆雞尾酒,8679,NAME
4,A01-010-00007,A01-010,0,M01,【芭達雅過癮馳騁】越野沙灘車半日遊,6950,NAME


In [7]:
prod_df.shape

(38950, 7)

In [8]:
prod_df=prod_df[['prod_oid','prod_desc','country_cd','city_cd','hot_sort','main_cat','property_type']]

In [9]:
prod_df.head()

,prod_oid,prod_desc,country_cd,city_cd,hot_sort,main_cat,property_type
0,9161,－ 開始時間：7:00am 或 4:00pm\n－ 出發地點：您於暹粒的下榻飯店\n－ 行程...,A01-014,A01-014-00003,0,M06,DESC
1,10689,"[{""tips"":{""checked"":""N""}},{""ins"":{""checked"":""N...",A01-004,A01-004-00001,0,M06,INC_DATA
2,10689,"[{""tips"":{""checked"":""N""}},{""ins"":{""checked"":""N...",A01-004,A01-004-00007,0,M06,INC_DATA
3,8679,【晚霞美酒】曼德勒烏本橋日落＆雞尾酒,A01-007,A01-007-00003,0,M06,NAME
4,6950,【芭達雅過癮馳騁】越野沙灘車半日遊,A01-010,A01-010-00007,0,M01,NAME


# Basic Info

In [203]:
all_prod=list(set(prod_df['prod_oid']))

In [231]:
prod_name=prod_df[prod_df['property_type']=='NAME'][['prod_oid','prod_desc']].groupby('prod_oid')['prod_desc'].first().reset_index()

In [232]:
prod_oid=prod_name['prod_oid'].tolist()
prod_name=prod_name['prod_desc'].tolist()
id_name=dict(zip(prod_oid,prod_name))

In [233]:
id_name[2247]

'【官方授權】日本環球影城™ 電子門票'

In [244]:
prod_hotsort=prod_df[['prod_oid','hot_sort']].groupby('prod_oid')['hot_sort'].first().reset_index()
prod_oid=prod_hotsort['prod_oid'].tolist()
prod_hotsort=prod_hotsort['hot_sort'].tolist()
id_hotsort=dict(zip(prod_oid,prod_hotsort))

In [245]:
id_hotsort[2247]

100

## Product City

In [140]:
city=prod_df[['prod_oid','city_cd']]

In [141]:
city=city.groupby('prod_oid')['city_cd'].apply(list).reset_index()

In [142]:
city.set_index('prod_oid',inplace=True)

In [147]:
city.loc[7,:]

city_cd    [A01-005-00001, A01-005-00001, A01-005-00001, ...
Name: 7, dtype: object

In [150]:
all_city=[]
for i in city['city_cd']:
    for e in i:
        if e not in all_city:
            all_city.append(e)

In [151]:
all_city

['A01-005-00001',
 'A01-001-00001',
 'A01-001-00006',
 'A01-001-00017',
 'A05-001-00001',
 'A01-005-00002',
 'A05-001-00005',
 'A05-001-00007',
 'A05-001-00006',
 'A01-006-00002',
 'A05-001-00008',
 'A01-006-00001',
 'A01-004-00001',
 'A01-004-00007',
 'A01-003-00001',
 'A05-001-00011',
 'A05-001-00010',
 'A05-001-00004',
 'A05-001-00003',
 'A01-007-00001',
 'A01-004-00002',
 'A01-009-00003',
 'A01-001-00019',
 'A01-009-00002',
 'A01-009-00004',
 'A01-010-00001',
 'A01-004-00005',
 'A01-001-00003',
 'A01-001-00008',
 'A01-010-00004',
 'A01-010-00006',
 'A01-001-00002',
 'A01-010-00002',
 'A01-011-00002',
 'A05-002-00002',
 'A01-013-00001',
 'A05-002-00001',
 'A05-001-00009',
 'A01-014-00003',
 'A05-001-00012',
 'A01-001-00012',
 'A01-001-00004',
 'A01-001-00005',
 'A03-004-00001',
 'A03-010-00001',
 'A03-008-00001',
 'A01-002-00001',
 'A03-004-00002',
 'A04-001-00002',
 'A04-001-00003',
 'A03-008-00005',
 'A05-001-00002',
 'A03-004-00005',
 'A03-011-00001',
 'A03-012-00001',
 'A01-002-

In [152]:
for i in all_city:
    city[i]=city['city_cd'].apply(lambda x: 1 if i in x else 0)

In [153]:
city.shape

(8097, 578)

In [156]:
city.drop('city_cd',axis=1,inplace=True)

In [157]:
city.shape

(8097, 577)

In [158]:
city.head()

,A01-005-00001,A01-001-00001,A01-001-00006,A01-001-00017,A05-001-00001,A01-005-00002,A05-001-00005,A05-001-00007,A05-001-00006,A01-006-00002,...,A03-020-00004,A03-004-00024,A03-021-00010,A03-009-00010,A03-009-00011,A03-006-00007,A03-020-00005,A06-001-00016,A05-001-00032,A06-001-00022
prod_oid,,,,,,,,,,,,,,,,,,,,,
7,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Product Category

In [177]:
prod_cat=prod_df[['prod_oid','main_cat']]

In [178]:
prod_cat=prod_cat.groupby('prod_oid').first().reset_index()

In [180]:
prod_cat.shape

(8097, 2)

In [181]:
all_cat=[]
for i in prod_cat['main_cat']:
    if i not in all_cat:
        all_cat.append(i)

In [182]:
for i in all_cat:
    prod_cat[i]=prod_cat['main_cat'].apply(lambda x: 1 if x==i else 0)

In [183]:
prod_cat.drop('main_cat',axis=1,inplace=True)

In [184]:
prod_cat.head()

,prod_oid,M01,M05,M03,M06,M02,M07,M08,M04
0,7,1,0,0,0,0,0,0,0
1,8,1,0,0,0,0,0,0,0
2,9,1,0,0,0,0,0,0,0
3,10,1,0,0,0,0,0,0,0
4,11,1,0,0,0,0,0,0,0


In [185]:
cat=prod_cat.set_index('prod_oid')

# Product Description

## Clean & Tokenize & Remove Stopwords

In [34]:
prod_desc=prod_df[['prod_oid','prod_desc']]

In [35]:
prod_desc.head()

,prod_oid,prod_desc
0,9161,－ 開始時間：7:00am 或 4:00pm\n－ 出發地點：您於暹粒的下榻飯店\n－ 行程...
1,10689,"[{""tips"":{""checked"":""N""}},{""ins"":{""checked"":""N..."
2,10689,"[{""tips"":{""checked"":""N""}},{""ins"":{""checked"":""N..."
3,8679,【晚霞美酒】曼德勒烏本橋日落＆雞尾酒
4,6950,【芭達雅過癮馳騁】越野沙灘車半日遊


In [36]:
prod_desc=prod_desc.groupby('prod_oid')['prod_desc'].apply(list).reset_index()

In [37]:
tokenizer = RegexpTokenizer(r'\w+')
prod_desc['prod_desc']=prod_desc['prod_desc'].apply(lambda x :tokenizer.tokenize(x.pop()))

In [14]:
def jiebait(lst):
    seglist=[]
    for word in lst:
        for w in jieba.cut(word):
            seglist.append(w)
    return seglist

In [38]:
prod_desc['prod_desc']=prod_desc['prod_desc'].apply(lambda x :jiebait(x))

In [40]:
prod_desc1=prod_desc.copy()

In [41]:
prod_desc.head()

,prod_oid,prod_desc
0,7,"[行程, 特色, 了解, 香港, 新界, 昔日, 歷史, 古, 蹟, 及, 轉變成, 現代,..."
1,8,"[夜色, 香港, 璀璨, 維港, 夜遊, 含, 晚餐]"
2,9,"[文化, 體驗, 香港, 傳統, 手工, 藝, 工作坊, 一日, 遊]"
3,10,"[香港, 風情, 大澳, 漁村, 深度, 遊]"
4,11,"[行程, 特色, 乘坐, 香港, 著名, 交通工具, 天星, 小輪, 叮叮, 車, 嘩, 啦..."


### Making Stopwords

In [17]:
all_words_in_desc=[]
for i in prod_desc['prod_desc']:
    for e in i:
        all_words_in_desc.append(e)

In [51]:
kkmost_common=[]
for (a,b) in Counter(all_words_in_desc).most_common(1000):
    kkmost_common.append(a)

In [19]:
kkmost_common

['的',
 '在',
 '行程',
 '時間',
 '您',
 '於',
 '00',
 '你',
 '和',
 '是',
 '請',
 '與',
 '1',
 '資訊',
 '地點',
 '為',
 '到',
 '2',
 '30',
 '3',
 '提供',
 '使用',
 '特色',
 '有',
 '出發',
 '集合',
 '可以',
 '介紹',
 '5',
 '後',
 '10',
 '分鐘',
 '小時',
 '4',
 '或',
 '站',
 '體驗',
 '酒店',
 '從',
 '上',
 '長',
 '8',
 '讓',
 '也',
 '來',
 '最',
 '服務',
 '7',
 '套餐',
 '不',
 '前往',
 '及',
 '票券',
 '月',
 '會',
 '約',
 '接送',
 '可',
 '時',
 '了',
 '飯店',
 '日',
 '兌換',
 '機場',
 '6',
 '人',
 '享受',
 '內',
 '等',
 '我們',
 '度',
 '旅客',
 '至',
 '9',
 '巴士',
 '之',
 '中',
 '都',
 '位',
 '重要',
 '12',
 '搭乘',
 '地址',
 '15',
 'meetPoint',
 '方式',
 '餐廳',
 '世界',
 '11',
 '年',
 '開始',
 '抵達',
 '景點',
 '號',
 'KKday',
 '當地',
 '選擇',
 '以',
 '欣賞',
 '20',
 '旅遊',
 '館',
 '回程',
 '地',
 'ー',
 '中心',
 '馬',
 '前',
 '個',
 '日本',
 '17',
 '料理',
 '憑證',
 '歷史',
 '參觀',
 '以及',
 '博物',
 '步行',
 '內容',
 '由',
 '而',
 'checked',
 '公園',
 '一個',
 '訂購',
 '以下',
 'の',
 '指定',
 '天',
 '包含',
 '製',
 '更',
 '接客',
 '城市',
 '開放',
 '遊',
 '文化',
 '就',
 '鬆',
 'N',
 '種類',
 '被',
 '18',
 '日期',
 '專業',
 '帶',
 '午餐',
 '大',
 '歲',
 '營業',
 '自行',


In [272]:
with open("kkmost_common.txt", "w") as output:
    output.write(str(kkmost_common))

In [273]:
lines = open("kkmost_common_edit.txt").read()

In [274]:
lines=lines.replace("'","")
kkstop=lines.split(',')

In [275]:
kkstop[:20]

['的',
 ' 在',
 ' 行程',
 ' 時間',
 ' 您',
 ' 於',
 ' 00',
 ' 你',
 ' 和',
 ' 是',
 ' 請',
 ' 與',
 ' 1',
 ' 資訊',
 ' 地點',
 ' 為',
 ' 到',
 ' 2',
 ' 30',
 ' 3']

### Remove Stopwords

In [32]:
def remove_kkstop(lst):
    result=[]
    for w in lst:
        if w not in kkstop:
            result.append(w)
    return result

In [276]:
prod_desc['prod_desc']=prod_desc['prod_desc'].apply(remove_kkstop)

In [277]:
#before 
prod_desc1.head()

,prod_oid,prod_desc
0,7,"[行程, 特色, 了解, 香港, 新界, 昔日, 歷史, 古, 蹟, 及, 轉變成, 現代,..."
1,8,"[夜色, 香港, 璀璨, 維港, 夜遊, 含, 晚餐]"
2,9,"[文化, 體驗, 香港, 傳統, 手工, 藝, 工作坊, 一日, 遊]"
3,10,"[香港, 風情, 大澳, 漁村, 深度, 遊]"
4,11,"[行程, 特色, 乘坐, 香港, 著名, 交通工具, 天星, 小輪, 叮叮, 車, 嘩, 啦..."


In [278]:
#after 
prod_desc.head()

,prod_desc
prod_oid,
7,"[香港, 新界, 昔日, 歷史, 古, 蹟, 轉變成, 現代, 故事, 探索, 原住, 居民..."
8,"[夜色, 香港, 璀璨, 維港, 夜遊, 晚餐]"
9,"[文化, 香港, 傳統, 手工, 藝, 工作坊, 一日]"
10,"[香港, 大澳, 漁村, 深度]"
11,"[香港, 交通工具, 天星, 小輪, 叮叮, 車, 嘩, 啦, 嘩, 啦, 舢舨, 環至, ..."


In [59]:
type(prod_desc['prod_desc'])

pandas.core.series.Series

In [52]:
all_desc=[]
for (a,b) in Counter(all_words_in_desc).most_common():
    if a not in kkstop:
        all_desc.append(a)

In [69]:
print ('Total Number of products:',len(prod_desc['prod_desc']))
print('Total Number of words:', len(all_desc))

Total Number of products: 8097
Total Number of words: 71025


In [ ]:
prod_desc.set_index('prod_oid',inplace=True)

### LDA 

In [56]:
from gensim import corpora

In [61]:
dictionary = corpora.Dictionary(prod_desc['prod_desc'])

In [62]:
corpus = [dictionary.doc2bow(doc) for doc in prod_desc['prod_desc']]

In [66]:
import gensim
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=10, id2word = dictionary, passes=20)

In [279]:
for i in ldamodel.print_topics(num_topics=10, num_words=5):
    print (i,'\n')

(0, '0.015*"機場" + 0.013*"新加坡" + 0.007*"香港" + 0.006*"票券" + 0.005*"WiFi"') 

(1, '0.021*"酒店" + 0.017*"機場" + 0.014*"飯店" + 0.014*"接送" + 0.011*"市區"') 

(2, '0.014*"按摩" + 0.011*"課程" + 0.007*"巴黎" + 0.005*"SPA" + 0.005*"泰式"') 

(3, '0.007*"公園" + 0.006*"潛水" + 0.005*"接送" + 0.005*"飯店" + 0.004*"海洋"') 

(4, '0.012*"表演" + 0.010*"票券" + 0.008*"搭乘" + 0.008*"門票" + 0.007*"入場"') 

(5, '0.010*"和服" + 0.008*"京都" + 0.007*"溫泉" + 0.006*"車站" + 0.006*"住宿"') 

(6, '0.010*"巴士" + 0.008*"日本" + 0.006*"札幌" + 0.005*"溫泉" + 0.004*"公園"') 

(7, '0.020*"滑雪" + 0.016*"반" + 0.014*"인" + 0.013*"납" + 0.011*"터"') 

(8, '0.010*"餐廳" + 0.007*"料理" + 0.007*"晚餐" + 0.005*"歷史" + 0.004*"飲品"') 

(9, '0.012*"해" + 0.012*"와" + 0.009*"馬" + 0.007*"부" + 0.006*"廣場"') 



## Product Name

In [102]:
name=prod_df[prod_df['property_type']=='NAME'][['prod_oid','prod_desc']]

In [103]:
name=name.groupby('prod_oid')['prod_desc'].first().reset_index()

In [104]:
name['prod_desc']=name['prod_desc'].apply(lambda x: jiebait(tokenizer.tokenize(x)))

In [105]:
name.head()

,prod_oid,prod_desc
0,7,"[香港, 新界, 文化, 遊, 走訪, 吉慶園, 林村, 許願樹, 沙田, 車公廟]"
1,8,"[夜色, 香港, 璀璨, 維港, 夜遊, 含, 晚餐]"
2,9,"[文化, 體驗, 香港, 傳統, 手工, 藝, 工作坊, 一日, 遊]"
3,10,"[香港, 風情, 大澳, 漁村, 深度, 遊]"
4,11,"[香港, 深度, 一日, 遊, 走訪, 中環, 石板, 街, 銅鑼灣, 香港, 仔避, 風塘]"


In [107]:
name['prod_desc']=name['prod_desc'].apply(remove_kkstop)

In [108]:
name.set_index('prod_oid',inplace=True)

## Jaccard Similarity -  Description & Name

In [110]:
def Product_Similarity(prod_oid_A,prod_oid_B,df):
    A=df.loc[prod_oid_A,'prod_desc']
    B=df.loc[prod_oid_B,'prod_desc']
    intersection_cardinality=len(set.intersection(*[set(A),set(B)]))
    union_cardinality=len(set.union(*[set(A),set(B)]))
    return float(intersection_cardinality)/union_cardinality

In [116]:
# product description 
Product_Similarity(2247,7781,prod_desc)

0.047619047619047616

In [115]:
# product name
Product_Similarity(2247,7781,name)

0.7142857142857143

## Cosine Distance - Category and City

In [118]:
from scipy import spatial

In [119]:
def Product_Distance(prod_oid_A,prod_oid_B,df):
    A=df.loc[prod_oid_A,:]
    B=df.loc[prod_oid_B,:]
    distance= spatial.distance.cosine(A,B)
    return distance 

In [194]:
# category 
Product_Distance(7781,9896,cat)

0.0

In [192]:
# city 
Product_Distance(7781,2247,city)

0.0

# Recommender 

In [205]:
import operator

In [196]:
product=input('Find Recommendation for:')

Find Recommendation for:2247


In [252]:
def get_rec(p):
    prod_dis=[]
    for i in all_prod:
        if i !=p:
            distance=Product_Distance(p,i,city)+Product_Distance(p,i,cat)
            prod_dis.append((i,distance))
    prod_dis.sort(key=operator.itemgetter(1),reverse=False)
    closest100=[]
    for i in range(100):
        closest100.append(prod_dis[i][0])
   
    prod_sim=[]
    for i in closest100:
        similarity=Product_Similarity(p,i,prod_desc)+Product_Similarity(p,i,name)
        prod_sim.append((i,similarity))
    prod_sim.sort(key=operator.itemgetter(1),reverse=True)
    
    rec_list=[]
    for i in range(20):
        rec_list.append((prod_sim[i][0],id_hotsort[prod_sim[i][0]]))
    
    rec_list.sort(key=operator.itemgetter(1),reverse=True)
    
    for i in range(10):
        print('Recommendation %d: %d %s ' %(i+1,rec_list[i][0],id_name[rec_list[i][0]]))

In [253]:
#with hotsort
get_rec(10999)

Recommendation 1: 2519 【富士山一日遊】五合目・河口湖・經典路線 
Recommendation 2: 9558 【東京賞楓包車一日遊】富士山・箱根・小江戶川越＆輕井澤 三條路線任選 
Recommendation 3: 10491 【箱根賞楓葉】富士山・箱根海盜船・御殿場OUTLET經典全制霸・東京出發 
Recommendation 4: 10539 【東京賞櫻一日遊】富士山河口湖櫻花．新倉富士淺間神社．御殿場OUTLET 
Recommendation 5: 9557 【富士山一日遊】五合目・平和公園・御殿場Outlet購物 
Recommendation 6: 10829 【富士山巴士一日遊】忍野八海＋富士溫泉＋御殿場Outlet 
Recommendation 7: 12319 【東京巴士一日遊】富士山河口湖・富岳風穴・鳴沢冰穴・忍野八海 
Recommendation 8: 11938 【東京巴士一日遊】河口湖薰衣草節・富士山五合目・採桃子吃到飽 
Recommendation 9: 18006 【富士山玩雪巴士一日遊】GRINPA冰雪樂園・伊豆水果公園・柿田川公園 
Recommendation 10: 10550 【富士山一日遊】多國語音導覽・河口湖、忍者村 


In [238]:
#without hotsort 
get_rec(10999)

Recommendation1: 10829 【富士山巴士一日遊】忍野八海＋富士溫泉＋御殿場Outlet 
Recommendation2: 9557 【富士山一日遊】五合目・平和公園・御殿場Outlet購物 
Recommendation3: 2519 【富士山一日遊】五合目・河口湖・經典路線 
Recommendation4: 12319 【東京巴士一日遊】富士山河口湖・富岳風穴・鳴沢冰穴・忍野八海 
Recommendation5: 11938 【東京巴士一日遊】河口湖薰衣草節・富士山五合目・採桃子吃到飽 
Recommendation6: 18006 【富士山玩雪巴士一日遊】GRINPA冰雪樂園・伊豆水果公園・柿田川公園 
Recommendation7: 10539 【東京賞櫻一日遊】富士山河口湖櫻花．新倉富士淺間神社．御殿場OUTLET 
Recommendation8: 10550 【富士山一日遊】多國語音導覽・河口湖、忍者村 
Recommendation9: 10900 【富士山一日遊】五合目、河口湖遊船 
Recommendation10: 18129 【東京巴士一日遊】寶石燈飾匯演・麗莎與卡斯柏小鎮・富士大石花露台・御殿場Outlet 


In [256]:
#with hotsort 
get_rec(2247)

Recommendation 1: 3995 【關西一卡通】ICOCA交通卡（含日本飯店 / 台灣香港郵寄） 
Recommendation 2: 10094 【官方主辦】戰國 The Real at 大阪城・Cool Japan 2017 
Recommendation 3: 7781 【官方授權】日本環球影城™ 門票＋快速® 通關 
Recommendation 4: 9911 【官方授權】日本環球影城™ 門票＋VIP入園手環＋HARUKAS展望台門票 
Recommendation 5: 9429 【官方授權】日本環球影城™ VIP入園手環＋阿倍野HARUKAS展望台門票 
Recommendation 6: 18015 【官方授權】日本環球影城™ 電子門票（香港旅客專屬） 
Recommendation 7: 13943 【官方授權】日本環球影城™ 1.5日實體門票・大阪領取 
Recommendation 8: 11279 【官方授權】日本環球影城®門票＋快速® 通關( 8/6 - 8/22 ) 
Recommendation 9: 11283 【官方授權】日本環球影城®門票＋快速® 通關 ( 6/25前 ) 
Recommendation 10: 9595 【USJ】日本環球影城門票＋快速通關６－逆轉世界－ 


In [239]:
#without hotsort 
get_rec(2247)

Recommendation1: 18015 【官方授權】日本環球影城™ 電子門票（香港旅客專屬） 
Recommendation2: 13943 【官方授權】日本環球影城™ 1.5日實體門票・大阪領取 
Recommendation3: 7781 【官方授權】日本環球影城™ 門票＋快速® 通關 
Recommendation4: 11279 【官方授權】日本環球影城®門票＋快速® 通關( 8/6 - 8/22 ) 
Recommendation5: 11283 【官方授權】日本環球影城®門票＋快速® 通關 ( 6/25前 ) 
Recommendation6: 9911 【官方授權】日本環球影城™ 門票＋VIP入園手環＋HARUKAS展望台門票 
Recommendation7: 9595 【USJ】日本環球影城門票＋快速通關６－逆轉世界－ 
Recommendation8: 17840 【跨年派對】日本環球影城™ 跨年派對門票 
Recommendation9: 9597 【USJ】日本環球影城門票＋萬聖節驚魂夜・快速通關５－恐怖實驗室－ 
Recommendation10: 9429 【官方授權】日本環球影城™ VIP入園手環＋阿倍野HARUKAS展望台門票 


In [257]:
#with hotsort 
get_rec(9912)

Recommendation 1: 11121 【曼谷潑水節獨家行程】安帕瓦水上市場＋在地寺廟潑水趣＋樹中廟 
Recommendation 2: 2151 【精華曼谷之旅】水上市場＋鐵道市集＋河濱夜市 
Recommendation 3: 10269 【經典曼谷一日遊】丹能沙朵水上市場＋美功鐵道市集＋河濱夜市 
Recommendation 4: 48 澳門一日遊（香港出發） 
Recommendation 5: 52 紐約出發，華盛頓觀光一日遊 
Recommendation 6: 4100 【深度安帕瓦一日遊】水上市場＋鐵道市場＋夜訪螢火蟲 
Recommendation 7: 6172 【曼谷單車美食】熱鬧的假日水上市集 
Recommendation 8: 5324 【曼谷親子遊必去】野生動物園＋巧克力村一日遊 
Recommendation 9: 1706 【曼谷精選一日遊】鐵道市場＋水上市場＋魔樹仙境＋龍波本廟＋河濱夜市 
Recommendation 10: 5313 【曼谷深度一日遊】丹能莎朵水上市場、美功鐵道市場、甜點博物館 


In [240]:
#without hotsort 
get_rec(9912)

Recommendation1: 10269 【經典曼谷一日遊】丹能沙朵水上市場＋美功鐵道市集＋河濱夜市 
Recommendation2: 2151 【精華曼谷之旅】水上市場＋鐵道市集＋河濱夜市 
Recommendation3: 5313 【曼谷深度一日遊】丹能莎朵水上市場、美功鐵道市場、甜點博物館 
Recommendation4: 6693 【清新曼谷一日遊】丹嫩莎朵水上市場＋玫瑰花園 
Recommendation5: 4100 【深度安帕瓦一日遊】水上市場＋鐵道市場＋夜訪螢火蟲 
Recommendation6: 11121 【曼谷潑水節獨家行程】安帕瓦水上市場＋在地寺廟潑水趣＋樹中廟 
Recommendation7: 4083 【精華水上市場】丹嫩莎朵＋鐵道市集＋河濱夜市＋泰拳公園＋樹中廟 
Recommendation8: 1706 【曼谷精選一日遊】鐵道市場＋水上市場＋魔樹仙境＋龍波本廟＋河濱夜市 
Recommendation9: 12038 【安帕瓦半日遊】造訪水上市場和鐵道市集奇景 
Recommendation10: 10272 【曼谷大城一日遊】造訪暹羅古城、水上市場、巧克力村 


Next Steps:
Incorporate Product Rating (# of stars)
